In [1]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset
from transformers import ElectraTokenizerFast, ElectraForSequenceClassification
from transformers import Trainer, TrainingArguments



def read_file(fname: str, correct_labels=False) -> pd.DataFrame:
    """Reads a filename, return df with text and labels.

    Args:
        fname (str): Filename to read
        correct_labels (bool, optional): If True, offensive instances get labeled 1
        and acceptable speech gets labeled 0. Else the labels remain unchanged.
        Defaults to False.

    Returns:
        pd.DataFrame: resulting dataframe with columns: text, labels
    """

    df = pd.read_table(fname, sep="\t", header=None, names="text,labels,role".split(","))
    if correct_labels:
        offensive_ids = df.labels != "Acceptable speech"
        df.labels[offensive_ids] = 1
        df.labels[~offensive_ids] = 0
        df["labels"] = df.labels.astype(int)
    df = df.drop(columns=["role"])
    return df

en_test, en_train = "../data/merged-en.test.tsv" , "../data/merged-en.train.tsv"
hr_test, hr_train = "../data/merged-hr.test.tsv" , "../data/merged-hr.train.tsv"
sl_test, sl_train = "../data/merged-sl.test.tsv",  "../data/merged-sl.train.tsv"

model_name = "classla/bcms-bertic"

train_df = read_file(hr_train, correct_labels=True)
test_df = read_file(hr_test, correct_labels=True)

train_texts, train_labels = train_df.text.values.tolist(), train_df.labels.values.tolist()
test_texts, test_labels = test_df.text.values.tolist(), test_df.labels.values.tolist()

class MergedHateDataset(Dataset):
    """ A dataset class for the merged hatespeech dataset (Frank)
    """    
    def __init__(self, encodings, labels) -> None:
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, index):
        item = {key: torch.tensor(val[index]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[index])
        return item
    def __len__(self):
        return len(self.labels)

tokenizer = ElectraTokenizerFast.from_pretrained(model_name)


train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

train_dataset = MergedHateDataset(train_encodings, train_labels)
test_dataset = MergedHateDataset(test_encodings, test_labels)

training_args = TrainingArguments(
    output_dir = "./outputs",
    num_train_epochs = 7,
    per_device_train_batch_size = 4,
    warmup_steps = 100,
    learning_rate = 3e-5,
    logging_dir = "./runs",
    overwrite_output_dir=True
)

model = ElectraForSequenceClassification.from_pretrained(model_name)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = test_dataset

)

trainer.train()

Some weights of the model checkpoint at classla/bcms-bertic were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at classla/bcms-bertic and are newly initialized: ['classifier.dense.weight', 'classifier.ou

Step,Training Loss
500,0.586100
1000,0.530500
1500,0.542500
2000,0.552200
2500,0.453400
3000,0.474600
3500,0.456100
4000,0.450900
4500,0.428000
5000,0.277000


Saving model checkpoint to ./outputs/checkpoint-500
Configuration saved in ./outputs/checkpoint-500/config.json
Model weights saved in ./outputs/checkpoint-500/pytorch_model.bin
Saving model checkpoint to ./outputs/checkpoint-1000
Configuration saved in ./outputs/checkpoint-1000/config.json
Model weights saved in ./outputs/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to ./outputs/checkpoint-1500
Configuration saved in ./outputs/checkpoint-1500/config.json
Model weights saved in ./outputs/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to ./outputs/checkpoint-2000
Configuration saved in ./outputs/checkpoint-2000/config.json
Model weights saved in ./outputs/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to ./outputs/checkpoint-2500
Configuration saved in ./outputs/checkpoint-2500/config.json
Model weights saved in ./outputs/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to ./outputs/checkpoint-3000
Configuration saved in ./outputs/checkpoint-3

TrainOutput(global_step=15491, training_loss=0.2286161774008664, metrics={'train_runtime': 2304.3218, 'train_samples_per_second': 26.887, 'train_steps_per_second': 6.723, 'total_flos': 1.630157165693952e+16, 'train_loss': 0.2286161774008664, 'epoch': 7.0})

# Saving the model

In [1]:
out_filename = "finetuned_models/HR_hate___classla_bcms-bertic_5"
model.save_pretrained(out_filename)
tokenizer.save_pretrained(out_filename)

NameError: name 'model' is not defined

# Repeated training

In [12]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset
from transformers import ElectraTokenizerFast, ElectraForSequenceClassification
from transformers import Trainer, TrainingArguments

in_filename = "finetuned_models/HR_hate___classla_bcms-bertic_10"

def read_file(fname: str, correct_labels=False) -> pd.DataFrame:
    """Reads a filename, return df with text and labels.

    Args:
        fname (str): Filename to read
        correct_labels (bool, optional): If True, offensive instances get labeled 1
        and acceptable speech gets labeled 0. Else the labels remain unchanged.
        Defaults to False.

    Returns:
        pd.DataFrame: resulting dataframe with columns: text, labels
    """

    df = pd.read_table(fname, sep="\t", header=None, names="text,labels,role".split(","))
    if correct_labels:
        offensive_ids = df.labels != "Acceptable speech"
        df.labels[offensive_ids] = 1
        df.labels[~offensive_ids] = 0
        df["labels"] = df.labels.astype(int)
    df = df.drop(columns=["role"])
    return df

en_test, en_train = "../data/merged-en.test.tsv" , "../data/merged-en.train.tsv"
hr_test, hr_train = "../data/merged-hr.test.tsv" , "../data/merged-hr.train.tsv"
sl_test, sl_train = "../data/merged-sl.test.tsv",  "../data/merged-sl.train.tsv"

model_name = in_filename

train_df = read_file(hr_train, correct_labels=True)
test_df = read_file(hr_test, correct_labels=True)

train_texts, train_labels = train_df.text.values.tolist(), train_df.labels.values.tolist()
test_texts, test_labels = test_df.text.values.tolist(), test_df.labels.values.tolist()

class MergedHateDataset(Dataset):
    """ A dataset class for the merged hatespeech dataset (Frank)
    """    
    def __init__(self, encodings, labels) -> None:
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, index):
        item = {key: torch.tensor(val[index]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[index])
        return item
    def __len__(self):
        return len(self.labels)

tokenizer = ElectraTokenizerFast.from_pretrained(model_name)


train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

train_dataset = MergedHateDataset(train_encodings, train_labels)
test_dataset = MergedHateDataset(test_encodings, test_labels)

training_args = TrainingArguments(
    output_dir = "./outputs",
    num_train_epochs = 5,
    per_device_train_batch_size = 4,
    warmup_steps = 100,
    learning_rate = 3e-5,
    logging_dir = "./runs",
    overwrite_output_dir=True
)

model = ElectraForSequenceClassification.from_pretrained(model_name)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = test_dataset

)

trainer.train()

Didn't find file finetuned_models/HR_hate___classla_bcms-bertic_10/added_tokens.json. We won't load it.
loading file finetuned_models/HR_hate___classla_bcms-bertic_10/vocab.txt
loading file finetuned_models/HR_hate___classla_bcms-bertic_10/tokenizer.json
loading file None
loading file finetuned_models/HR_hate___classla_bcms-bertic_10/special_tokens_map.json
loading file finetuned_models/HR_hate___classla_bcms-bertic_10/tokenizer_config.json
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file finetuned_models/HR_hate___classla_bcms-bertic_10/config.json
Model config ElectraConfig {
  "_name_or_path": "finetuned_models/HR_hate___classla_bcms-bertic_9",
  "architectures": [
    "ElectraForSequenceClassification"
  ],
  "at

Step,Training Loss
500,0.036000
1000,0.046300
1500,0.019000
2000,0.033200
2500,0.030300
3000,0.042400
3500,0.003100
4000,0.015400
4500,0.026800
5000,0.032500


Saving model checkpoint to ./outputs/checkpoint-500
Configuration saved in ./outputs/checkpoint-500/config.json
Model weights saved in ./outputs/checkpoint-500/pytorch_model.bin
Saving model checkpoint to ./outputs/checkpoint-1000
Configuration saved in ./outputs/checkpoint-1000/config.json
Model weights saved in ./outputs/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to ./outputs/checkpoint-1500
Configuration saved in ./outputs/checkpoint-1500/config.json
Model weights saved in ./outputs/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to ./outputs/checkpoint-2000
Configuration saved in ./outputs/checkpoint-2000/config.json
Model weights saved in ./outputs/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to ./outputs/checkpoint-2500
Configuration saved in ./outputs/checkpoint-2500/config.json
Model weights saved in ./outputs/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to ./outputs/checkpoint-3000
Configuration saved in ./outputs/checkpoint-3

TrainOutput(global_step=11065, training_loss=0.020669079484090618, metrics={'train_runtime': 1631.2458, 'train_samples_per_second': 27.13, 'train_steps_per_second': 6.783, 'total_flos': 1.16439797549568e+16, 'train_loss': 0.020669079484090618, 'epoch': 5.0})

In [13]:
out_filename_after_additional_training = "finetuned_models/HR_hate___classla_bcms-bertic_11"
model.save_pretrained(out_filename_after_additional_training)
tokenizer.save_pretrained(out_filename_after_additional_training)

Configuration saved in finetuned_models/HR_hate___classla_bcms-bertic_11/config.json
Model weights saved in finetuned_models/HR_hate___classla_bcms-bertic_11/pytorch_model.bin
tokenizer config file saved in finetuned_models/HR_hate___classla_bcms-bertic_11/tokenizer_config.json
Special tokens file saved in finetuned_models/HR_hate___classla_bcms-bertic_11/special_tokens_map.json


('finetuned_models/HR_hate___classla_bcms-bertic_11/tokenizer_config.json',
 'finetuned_models/HR_hate___classla_bcms-bertic_11/special_tokens_map.json',
 'finetuned_models/HR_hate___classla_bcms-bertic_11/vocab.txt',
 'finetuned_models/HR_hate___classla_bcms-bertic_11/added_tokens.json',
 'finetuned_models/HR_hate___classla_bcms-bertic_11/tokenizer.json')